In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import spacy
import numpy as np

In [ ]:
!python -m spacy download en_core_web_md

2023-11-27 09:29:29.444674: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-27 09:29:29.444748: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-27 09:29:29.444796: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-27 09:29:31.331714: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 17.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
nlp = spacy.load("en_core_web_md")

def scrape_wikipedia(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    paragraphs = soup.find_all('p')
    text = ' '.join([p.get_text() for p in paragraphs])
    return text

def create_text_embeddings(text):
    doc = nlp(text)
    return doc.vector

In [ ]:
csv_file_path = '/content/merged_dataset.csv'
df = pd.read_csv(csv_file_path)

columns_to_drop = ['status', 'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time',
                   'quali_date', 'quali_time', 'sprint_date', 'sprint_time',
                   'circuitRef', 'name_circuit', 'location', 'country', 'constructorRef', 'name',
                   'nationality_constructor', 'driverRef', 'number_driver',
                   'forename', 'surname', 'dob', 'nationality_driver'
                   ]
df = df.drop(columns=columns_to_drop, axis=1)
df.dropna(inplace=True)


<ipython-input-17-be2b8f66e337>:3: DtypeWarning: Columns (4,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path)


In [ ]:
url_columns = [col for col in df.columns if 'url' in col.lower()]
max_embedding_length = 0

for col in url_columns:
    for i in range(max_embedding_length):
        df[f"{col}_embedding_{i}"] = np.nan

embeddings_dict = {}
for index, row in df.iterrows():
    for col in url_columns:
        url = row[col]
        if pd.notnull(url) and url not in embeddings_dict:
            try:
                text = scrape_wikipedia(url)
                embeddings = create_text_embeddings(text)
                embeddings_dict[url] = embeddings
            except Exception as e:
                print(f"Error processing URL {url}: {str(e)}")

for index, row in df.iterrows():
    for col in url_columns:
        url = row[col]
        if url in embeddings_dict:
            embeddings = embeddings_dict[url]
            for i, embedding_value in enumerate(embeddings):
                df.at[index, f"{col}_embedding_{i}"] = embedding_value

df.to_csv('output_with_embeddings.csv', index=False)

<ipython-input-18-ed52eca90ff6>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.at[index, f"{col}_embedding_{i}"] = embedding_value
<ipython-input-18-ed52eca90ff6>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.at[index, f"{col}_embedding_{i}"] = embedding_value
<ipython-input-18-ed52eca90ff6>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get 